In [3]:
import os
import glob
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import losses
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential
print("Tensorflow version:", tf.__version__)

from sklearn.model_selection import train_test_split

Tensorflow version: 2.14.0


In [4]:
# Tentukan path ke folder dataset
folder_path = 'Dataset/'

# Buat pola untuk mencocokkan file CSV di dalam folder
file_pattern = '*.csv'

# Gunakan modul glob untuk mendapatkan daftar file yang sesuai dengan pola
file_list = glob.glob(folder_path + file_pattern)

# Buat list kosong untuk menyimpan DataFrame dari setiap file CSV
dfs = []

a = 0
# Loop melalui setiap file dan baca sebagai DataFrame, lalu tambahkan ke list
for file in file_list:
    df = pd.read_csv(file)
    dfs.append(df)

# Gabungkan semua DataFrame menjadi satu
merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df[['text','status']]

# Mengganti nilai 'berbahaya' menjadi -1 dan 'aman' menjadi 1
merged_df['status_number'] = merged_df['status'].replace({'bahaya': -1, 'aman': 1})

In [5]:
merged_df.tail(5)

,text,status,status_number
370,news academic research unja acceptance student...,aman,1
371,join IDN Times Quiz News Business Sport Tech H...,aman,1
372,sign Ooa Puberty manage Faculty Medicine Brawi...,aman,1
373,home zenius insights campus subjects exams tip...,aman,1
374,"children, examples school materials online cou...",aman,1


In [6]:
# Mengambil indeks DataFrame
indices = merged_df.index.values

# Mengacak indeks menggunakan numpy.random.shuffle
np.random.shuffle(indices)

# Mengindeks ulang DataFrame dengan indeks yang diacak
df = merged_df.iloc[indices].reset_index(drop=True)

# Menampilkan DataFrame setelah diacak
df.head(10)

,text,status,status_number
0,searchsvgfillfff spicevids free brazzers libra...,bahaya,-1
1,select area website search study guides homewo...,aman,1
2,get hairy pussy updates daily see exclusive mo...,bahaya,-1
3,main university california santa barbara give ...,aman,1
4,obokozu japanese beauty sucking edging boyfrie...,bahaya,-1
5,home adventures planet people animals bride lo...,aman,1
6,stars get sankaku plus forum app hide recent p...,bahaya,-1
7,join bang bros home new videos top girls websi...,bahaya,-1
8,private attackers might trying steal informati...,bahaya,-1
9,main navigation online residential athletics a...,aman,1


# <h2><b>Modelling

In [7]:
# convert airline_seentiment to numeric
sentiment_label = df.status_number.factorize()
sentiment_label

(array([0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

teks = df.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(teks)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(teks)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [9]:
print(tokenizer.word_index)

{'videos': 1, 'free': 2, 'search': 3, 'new': 4, 'home': 5, 'get': 6, 'sex': 7, 'site': 8, 'porn': 9, 'live': 10, 'website': 11, 'learn': 12, 'news': 13, 'research': 14, 'students': 15, 'categories': 16, 'information': 17, 'best': 18, 'sign': 19, 'main': 20, 'top': 21, 'online': 22, 'community': 23, 'university': 24, 'big': 25, 'find': 26, 'join': 27, 'years': 28, 'campus': 29, 'help': 30, 'life': 31, 'use': 32, 'academics': 33, 'explore': 34, 'terms': 35, 'view': 36, 'admissions': 37, 'student': 38, 'pornstars': 39, 'hot': 40, 'models': 41, 'world': 42, 'read': 43, 'anal': 44, 'enter': 45, 'privacy': 46, 'services': 47, 'latest': 48, 'age': 49, 'today': 50, 'experience': 51, 'sites': 52, 'alumni': 53, 'time': 54, 'adult': 55, 'contact': 56, 'games': 57, 'english': 58, 'ass': 59, 'popular': 60, 'give': 61, 'visit': 62, 'amateur': 63, 'support': 64, 'learning': 65, 'cams': 66, 'policy': 67, 'content': 68, 'faculty': 69, 'hd': 70, 'parents': 71, 'education': 72, 'paling': 73, 'tits': 74, 

In [10]:
print(teks[0])
print(encoded_docs[0])

searchsvgfillfff spicevids free brazzers library tags watch full length videos pornstar hdporn big tits ass keras trailers dick milf blonde brunette blowjob babe teen threesome popular women role play rough sex small hd reality lesbian step fantasy redaksi head latina see categories tagged bigdick boobs pornohub mgvideos porhub bigass butt anal bigcock fake jordi enp takes picture isabelle deltores jerks watching iiia naughty gizelle blanco buys new oil kendr
[2693, 1210, 2, 653, 542, 125, 149, 105, 782, 1, 954, 2694, 25, 74, 59, 159, 2695, 280, 84, 111, 116, 160, 179, 87, 321, 60, 161, 1211, 117, 955, 7, 180, 70, 654, 126, 322, 1212, 956, 1691, 468, 100, 16, 1692, 2696, 281, 2697, 2698, 2699, 2700, 783, 44, 2701, 2702, 957, 2703, 401, 784, 2704, 2705, 2706, 1213, 2707, 655, 1693, 2708, 2709, 4, 1214, 2710]


In [11]:
print(padded_sequence[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0 2693 1210    2  653  542  125  149  105  782
    1  954 2694   25   74   59  159 2695  280   84  111  116  160  179
   87  321   60  161 1211  117  955    7  180   70  654  126  322 1212
  956 1691  468  100   16 1692 2696  281 2697 2698 2699 2700  783   44
 2701 2702  957 2703  401  784 2704 2705 2706 1213 2707  655 1693 2708
 2709 

<h3><b>Build model

In [12]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import Embedding
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length,
                                     input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',
                           metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           218240    
                                                                 
 spatial_dropout1d (Spatial  (None, 200, 32)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 234891 (917.54 KB)
Trainable params: 234891 (917.54 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [13]:
history = model.fit(padded_sequence,sentiment_label[0],
                  validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
10/10 [==============================] - 11s 411ms/step - loss: 0.6907 - accuracy: 0.5767 - val_loss: 0.6986 - val_accuracy: 0.3733
Epoch 2/5
10/10 [==============================] - 3s 335ms/step - loss: 0.6822 - accuracy: 0.5600 - val_loss: 0.7078 - val_accuracy: 0.3733
Epoch 3/5
10/10 [==============================] - 3s 322ms/step - loss: 0.6619 - accuracy: 0.5567 - val_loss: 0.7121 - val_accuracy: 0.3733
Epoch 4/5
10/10 [==============================] - 3s 320ms/step - loss: 0.5408 - accuracy: 0.6267 - val_loss: 0.4806 - val_accuracy: 0.9467
Epoch 5/5
10/10 [==============================] - 3s 312ms/step - loss: 0.3313 - accuracy: 0.9400 - val_loss: 0.2936 - val_accuracy: 0.9867


In [17]:
# test_word ="warning adults woman sex life gay video xxx violet cum smooth lesbian sex main skyla alex moan mouth ending af"
test_word ="porn network bravo hell moms plus porno free pornotv sign bravotubenet come find best web home newest popular categories models live girls hd last viewed videos added hours ago smoking hot chick gets pinkhaired japanese chick uterus blasted iae white milf wife meets big iea amateur girlfriend double ae kinky chicks love feeling rough interracial bdms fucking makes slutty lana violet cum smooth lesbian sex main skyla alex moan mouth ending af"
tw = tokenizer.texts_to_sequences([test_word])
tw = pad_sequences(tw,maxlen=200)
prediction = int(model.predict(tw).round().item())
sentiment_label[1][prediction]

1/1 [==============================] - 0s 60ms/step


-1